# Compile the results on the MOF case study 

In [480]:
from glob import glob
from pathlib import Path
import matplotlib.pyplot as plt
plt.style.use(['science', 'nature'])
import pandas as pd 
from fastcore.helpers import load_pickle
import numpy as np 
from gpt3forchem.helpers import get_else_nan
from gpt3forchem.api_wrappers import extract_regression_prediction
from gpt3forchem.output import get_regression_metrics

In [481]:
def compile_table_row(row):
    template =  "\\num⁍ {accuracy} \\pm {accuracy_std} ⁌ &  \\num⁍ {f1_micro} \\pm {f1_micro_std}  ⁌ & \\num⁍ {f1_macro} \\pm {f1_macro_std} ⁌\\\\"
    row = row.round(2)
    string = template.format(
        accuracy=row['accuracy']['mean'].values[0],
        accuracy_std=row['accuracy']['std'].values[0],
        f1_micro=row['f1_micro']['mean'].values[0],
        f1_micro_std=row['f1_micro']['std'].values[0],
        f1_macro=row['f1_macro']['mean'].values[0],
        f1_macro_std=row['f1_macro']['std'].values[0]
    )

    return string.replace("⁍", "{").replace("⁌", "}")

def compile_table_baseline_row(row):
    template =  "\\num⁍ {accuracy} \\pm {accuracy_std} ⁌ &  \\num⁍ {f1_micro} \\pm {f1_micro_std}  ⁌ & \\num⁍ {f1_macro} \\pm {f1_macro_std} ⁌\\\\"
    row = row.round(2)
    string = template.format(
        accuracy=row['baseline_accuracy']['mean'].values[0],
        accuracy_std=row['baseline_accuracy']['std'].values[0],
        f1_micro=row['baseline_f1_micro']['mean'].values[0],
        f1_micro_std=row['baseline_f1_micro']['std'].values[0],
        f1_macro=row['baseline_f1_macro']['mean'].values[0],
        f1_macro_std=row['baseline_f1_macro']['std'].values[0]
    )

    return string.replace("⁍", "{").replace("⁌", "}")

## QMOF 

### Classification 

In [482]:
all_qmof_classification_res = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220913_mof_classification/*.pkl')

In [490]:
compiled_qmof_classification_data = []
compiled_qmof_baseline_data = []


compiled_qmof_classification_data_skip_hyperopt = []
compiled_qmof_baseline_data_skip_hyperopt = []


# ToDo: handle the case in which the baseline used the wrong test data - skip them if we have both numerics and string
for filename in all_qmof_classification_res:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    try:
        if len(loaded_res["baseline_cm"].classes) == 5:
            baseline_res = {
                "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
                "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
                "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
                "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
                "train_size": loaded_res["train_size"],
                "representation": loaded_res["representation"],
                "target": loaded_res["target"],
            }
        else: 
            raise ValueError
    except Exception: 
           baseline_res = {
                "baseline_accuracy": np.nan,
                "baseline_f1_macro": np.nan,
                "baseline_f1_micro": np.nan,
                "baseline_mcc": np.nan,
                "train_size": loaded_res["train_size"],
                "representation": loaded_res["representation"],
                "target": loaded_res["target"],
            } 
    if 'skip' in filename:
        compiled_qmof_classification_data.append(res)
        compiled_qmof_baseline_data_skip_hyperopt.append(baseline_res)
    else:
        compiled_qmof_classification_data.append(res)
        compiled_qmof_baseline_data.append(baseline_res)


In [491]:
compiled_qmof_classification_data = pd.DataFrame(compiled_qmof_classification_data)
compiled_qmof_baseline_data = pd.DataFrame(compiled_qmof_baseline_data)

compiled_qmof_classification_data_skip_hyperopt = pd.DataFrame(compiled_qmof_classification_data_skip_hyperopt)
compiled_qmof_baseline_data_skip_hyperopt = pd.DataFrame(compiled_qmof_baseline_data_skip_hyperopt)

In [492]:
compiled_qmof_baseline_data_skip_hyperopt.groupby(['target', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_99655/3485326642.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_qmof_baseline_data_skip_hyperopt.groupby(['target', 'train_size']).agg(['mean', 'std'])


baseline_accuracy            \
                                                mean       std   
target                  train_size                               
outputs.pbe.bandgap_cat 10                  0.709063  0.013206   
                        50                  0.735119  0.005988   
                        100                 0.740346  0.005675   
                        200                 0.751736  0.002329   
                        500                 0.771794  0.003894   
                        1000                0.787106  0.002284   
                        2000                0.802023  0.003151   
                        3000                0.813426  0.006305   

                                   baseline_f1_macro            \
                                                mean       std   
target                  train_size                               
outputs.pbe.bandgap_cat 10                  0.185662  0.013295   
                        50                  0.229145  0.015604   
                        100                 0.238457  0.017350   
                        200                 0.282819  0.015911   
                        500                 0.357373  0.023660   
                        1000                0.404268  0.025026   
                        2000                0.452838  0.036692   
                        3000                0.491458  0.049201   

                                   baseline_f1_micro           baseline_mcc  \
                                                mean       std         mean   
target                  train_size                                            
outputs.pbe.bandgap_cat 10                  0.272657  0.033015     0.003969   
                        50                  0.337797  0.014969     0.042884   
                        100                 0.350865  0.014188     0.055555   
                        200                 0.379341  0.005823     0.103591   
                        500                 0.429485  0.009736     0.177773   
                        1000                0.467765  0.005709     0.232906   
                        2000                0.505057  0.007876     0.287331   
                        3000                0.533565  0.015762     0.329239   

                                              
                                         std  
target                  train_size            
outputs.pbe.bandgap_cat 10          0.025721  
                        50          0.018518  
                        100         0.023818  
                        200         0.010545  
                        500         0.014886  
                        1000        0.009743  
                        2000        0.012027  
                        3000        0.021908

In [493]:
compiled_qmof_baseline_data_skip_hyperopt

,baseline_accuracy,baseline_f1_macro,baseline_f1_micro,baseline_mcc,train_size,representation,target
0,0.723307,0.198947,0.308267,0.011310,10,chemical_name,outputs.pbe.bandgap_cat
1,0.746867,0.255328,0.367168,0.082474,100,chemical_name,outputs.pbe.bandgap_cat
2,0.739223,0.263108,0.348057,0.062488,50,chemical_name,outputs.pbe.bandgap_cat
3,0.753487,0.250687,0.383718,0.107179,200,chemical_name,outputs.pbe.bandgap_cat
4,0.733144,0.218896,0.332861,0.021617,100,chemical_name,outputs.pbe.bandgap_cat
...,...,...,...,...,...,...,...
80,0.807253,0.517046,0.518132,0.307009,2000,chemical_name,outputs.pbe.bandgap_cat
81,0.820096,0.470443,0.550239,0.351770,3000,chemical_name,outputs.pbe.bandgap_cat
82,0.775248,0.347948,0.438120,0.188616,500,chemical_name,outputs.pbe.bandgap_cat
83,0.749702,0.254978,0.374256,0.086916,100,chemical_name,outputs.pbe.bandgap_cat


In [494]:
compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_99655/4159097433.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_qmof_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


accuracy            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.735467  0.021914   
                                               50          0.768800  0.041899   
                                               100         0.818133  0.014048   
                                               200         0.842667  0.011210   
                                               500         0.860800  0.007920   
                                               1000        0.885022  0.016213   
                                               2000        0.900356  0.019524   
                                               3000        0.916400  0.008485   
                        info.mofid.mofid_clean 10          0.717433  0.037845   
                                               50          0.763985  0.024521   
                                               100         0.845259  0.058360   
                                               200         0.830480  0.046759   
                                               500         0.870667  0.007668   
                                               1000        0.883387  0.007723   
                                               2000        0.900533  0.009272   
                                               3000        0.908000  0.015839   

                                                           f1_macro            \
                                                               mean       std   
target                  representation         train_size                       
outputs.pbe.bandgap_cat chemical_name          10          0.194741  0.038253   
                                               50          0.182454  0.098772   
                                               100         0.292088  0.052740   
                                               200         0.368055  0.016604   
                                               500         0.518580  0.079594   
                                               1000        0.505309  0.067886   
                                               2000        0.575970  0.072864   
                                               3000        0.664919  0.061100   
                        info.mofid.mofid_clean 10          0.141765  0.090571   
                                               50          0.174914  0.070117   
                                               100         0.293975  0.158088   
                                               200         0.368876  0.096230   
                                               500         0.501428  0.014631   
                                               1000        0.512295  0.044672   
                                               2000        0.577524  0.123864   
                                               3000        0.605338  0.035133   

                                                           f1_micro            
                                                               mean       std  
target                  representation         train_size                      
outputs.pbe.bandgap_cat chemical_name          10          0.338667  0.054784  
                                               50          0.422000  0.104747  
                                               100         0.545333  0.035119  
                                               200         0.606667  0.028024  
                                               500         0.652000  0.019799  
                                               1000        0.694667  0.039716  
                                               2000        0.738000  0.026907  
                                               3000        0.791000  0.021213  
                        info.mofid.mofid_clean 10          0.252500  0.168312  
                   

In [489]:
compiled_qmof_baseline_data[['target', 'train_size', 'baseline_accuracy', 'baseline_f1_micro', 'baseline_f1_macro']].groupby(['target', 'train_size']).agg(['std', 'mean'])

baseline_accuracy            \
                                                 std      mean   
target                  train_size                               
outputs.pbe.bandgap_cat 10                       NaN       NaN   
                        50                       NaN  0.747274   
                        100                      NaN       NaN   
                        200                      NaN       NaN   
                        500                      NaN       NaN   
                        1000                     NaN       NaN   
                        2000                     NaN       NaN   
                        3000                     NaN       NaN   

                                   baseline_f1_micro            \
                                                 std      mean   
target                  train_size                               
outputs.pbe.bandgap_cat 10                       NaN       NaN   
                        50                       NaN  0.368186   
                        100                      NaN       NaN   
                        200                      NaN       NaN   
                        500                      NaN       NaN   
                        1000                     NaN       NaN   
                        2000                     NaN       NaN   
                        3000                     NaN       NaN   

                                   baseline_f1_macro            
                                                 std      mean  
target                  train_size                              
outputs.pbe.bandgap_cat 10                       NaN       NaN  
                        50                       NaN  0.218125  
                        100                      NaN       NaN  
                        200                      NaN       NaN  
                        500                      NaN       NaN  
                        1000                     NaN       NaN  
                        2000                     NaN       NaN  
                        3000                     NaN       NaN

### Regression 

In [398]:
qmof_regression_results = glob('/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/*.pkl')

In [399]:
load_pickle(qmof_regression_results[0])

{'model_type': 'ada',
 'train_set_size': 2000,
 'prefix': '',
 'train_size': 2000,
 'test_size': 1627,
 'metrics': {'r2': 0.7192669331671564,
  'max_error': 2.5827999999999998,
  'mean_absolute_error': 0.435709634,
  'mean_squared_error': 0.374205485176178},
 'completions': {'choices': [<OpenAIObject at 0x2b4fb1ea0> JSON: {
     "finish_reason": "length",
     "index": 0,
     "logprobs": null,
     "text": " 0.9595@@@4.936"
   },
   <OpenAIObject at 0x2b4fb1e00> JSON: {
     "finish_reason": "length",
     "index": 1,
     "logprobs": null,
     "text": " 1.934@@@+1.9"
   },
   <OpenAIObject at 0x2b4fb1cc0> JSON: {
     "finish_reason": "length",
     "index": 2,
     "logprobs": null,
     "text": " 0.49@@@@@@ 0.49"
   },
   <OpenAIObject at 0x17f631720> JSON: {
     "finish_reason": "length",
     "index": 3,
     "logprobs": null,
     "text": " 0.03934@@@0.49"
   },
   <OpenAIObject at 0x17f6319f0> JSON: {
     "finish_reason": "length",
     "index": 4,
     "logprobs": null,
   

In [400]:
def metrics_ignoring_nan(valid_file, completions): 
    valid_frame = pd.read_json(valid_file, lines=True, orient='records')
    true = np.array([
            float(valid_frame.iloc[i]["completion"].split("@")[0])
            for i in range(len(valid_frame))
        ])
    prd = np.array([extract_regression_prediction(completions, i) for i in range(len(completions['choices']))])

    true = true[:len(prd)]
    prd_no_nan = prd[~np.isnan(prd)]
    true_no_nan = true[~np.isnan(prd)]
    
    metrics = get_regression_metrics(true_no_nan, prd_no_nan)
    return metrics

In [401]:
compiled_regression_results = []
compiled_regression_baseline_results = []


# ToDo: handle nan metrics (ignore nan in the computation of the metrics)
for res_file in qmof_regression_results:
    if '3000' in res_file:
        print(res_file)
    try:
        res = load_pickle(res_file)
        res_dict = {
            'r2': res['metrics']['r2'],
            'max_error': res['metrics']['max_error'],
            'mean_absolute_error': res['metrics']['mean_absolute_error'],
            'mean_squared_error': res['metrics']['mean_squared_error'],
        }
        if np.isnan(res_dict['r2']):
            # recompute metrics
            print('recomputing metrics')
            res_dict = metrics_ignoring_nan(res['valid_filename'], res['completions'])
        res_dict['train_size'] = res['train_size']
        res_dict['representation'] = res['representation']
        res_dict['target'] = res['target']
        compiled_regression_results.append(res_dict)
        
        baseline_res_dict = res['baseline_metrics']
        baseline_res_dict['train_size'] = res['train_size']
        baseline_res_dict['target'] = res['target']

        
        compiled_regression_baseline_results.append(baseline_res_dict)
    
    except Exception as e: 
        pass
    
 

/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/2022-09-16-08-16-14_results_mof_regression_3000__ada_info.mofid.mofid_clean_outputs.pbe.bandgap.pkl
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/2022-09-23-08-55-39_results_mof_regression_3000__ada_info.mofid.mofid_clean_outputs.pbe.bandgap.pkl
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/2022-09-25-22-10-44_results_mof_regression_3000__ada_chemical_name_outputs.pbe.bandgap.pkl
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/2022-09-26-00-19-44_results_mof_regression_3000__ada_chemical_name_outputs.pbe.bandgap.pkl
/Users/kevinmaikjablonka/git/kjappelbaum/gpt3forchem/experiments/results/20220915_mof_regression/2022-09-26-04-27-25_results_mof_regression_3000__ada_chemical_name_outputs.pbe.bandgap.pkl
recomputing metrics
/Users/kevinmaikjablon

In [402]:
compiled_regression_results = pd.DataFrame(compiled_regression_results)
compiled_regression_baseline_results = pd.DataFrame(compiled_regression_baseline_results)

In [403]:
compiled_regression_results.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std', 'count'])

r2            \
                                                           mean       std   
target              representation         train_size                       
outputs.pbe.bandgap chemical_name          10         -1.053107  0.670171   
                                           50         -0.582105  0.468003   
                                           100         0.244085  0.165655   
                                           200         0.434591  0.115752   
                                           500         0.552918  0.032685   
                                           1000        0.617889  0.035222   
                                           2000        0.668723  0.048312   
                                           3000        0.726500  0.006635   
                    info.mofid.mofid_clean 10         -0.594362  0.507920   
                                           50         -0.327317  0.305812   
                                           100        -0.277853  0.424610   
                                           200         0.391439  0.099010   
                                           500         0.578588  0.037843   
                                           1000        0.543437  0.096453   
                                           2000        0.629173  0.073341   
                                           3000        0.699638  0.020831   

                                                            max_error  \
                                                      count      mean   
target              representation         train_size                   
outputs.pbe.bandgap chemical_name          10             6  4.504935   
                                           50             5  3.755916   
                                           100            5  3.450556   
                                           200            5  3.042948   
                                           500            5  3.197736   
                                           1000           5  3.097620   
                                           2000           5  2.921456   
                                           3000           5  3.332256   
                    info.mofid.mofid_clean 10             5  3.459420   
                                           50             6  3.614013   
                                           100            6  3.739493   
                                           200            6  3.713433   
                                           500            5  3.215900   
                                           1000           4  3.088475   
                                           2000           4  2.815765   
                                           3000           4  2.943025   

                                                                       \
                                                            std count   
target              representation         train_size                   
outputs.pbe.bandgap chemical_name          10          0.788691     6   
                                           50          0.849512     5   
                                           100         0.611235     5   
                                           200         0.300688     5   
                                           500         0.304235     5   
                                           1000        0.309496     5   
                                           2000        0.074627     5   
                                           3000        0.420635     5   
                    info.mofid.mofid_clean 10          0.439433     5   
                                           50          0.589225     6   
                                           100         0.473727     6   
                                           200         0.544137     6   
                                           500         0.710886     5   
                                         

In [404]:
compiled_regression_baseline_results.groupby(['target', 'train_size']).agg(['mean', 'std'])

r2           max_error       \
                                    mean       std      mean  std   
target              train_size                                      
outputs.pbe.bandgap 10         -1.866861  0.000411  5.688000  0.0   
                    50         -0.158858       NaN  4.121102  NaN   
                    100        -0.113880       NaN  4.027462  NaN   
                    200        -0.150069       NaN  3.622514  NaN   
                    500        -0.415040       NaN  4.863390  NaN   
                    1000             NaN       NaN       NaN  NaN   
                    2000             NaN       NaN       NaN  NaN   
                    3000             NaN       NaN       NaN  NaN   

                               mean_absolute_error            \
                                              mean       std   
target              train_size                                 
outputs.pbe.bandgap 10                    1.645561  0.000322   
                    50                    1.047274       NaN   
                    100                   1.025447       NaN   
                    200                   1.042389       NaN   
                    500                   1.162051       NaN   
                    1000                       NaN       NaN   
                    2000                       NaN       NaN   
                    3000                       NaN       NaN   

                               mean_squared_error            
                                             mean       std  
target              train_size                               
outputs.pbe.bandgap 10                   3.948074  0.001489  
                    50                   1.597440       NaN  
                    100                  1.532360       NaN  
                    200                  1.587646       NaN  
                    500                  1.959075       NaN  
                    1000                      NaN       NaN  
                    2000                      NaN       NaN  
                    3000                      NaN       NaN

## CoRE MOF

In [405]:
all_core_results = glob('results/20220914_mof_classification/*pkl')

In [406]:
compiled_core_classification_data = []
compiled_core_baseline_data = []

compiled_core_classification_data_skip_hyperopt = []
compiled_core_baseline_data_skip_hyperopt = []

for filename in all_core_results:
    loaded_res = load_pickle(filename)

    res = {
        "accuracy": get_else_nan(loaded_res["cm"], "ACC_Macro"),
        "f1_macro": get_else_nan(loaded_res["cm"], "F1_Macro"),
        "f1_micro": get_else_nan(loaded_res["cm"], "F1_Micro"),
        "train_size": loaded_res["train_size"],
        "mcc": get_else_nan(loaded_res["cm"], "Overall_MCC"),
        "representation": loaded_res["representation"],
        "target": loaded_res["target"],
    }

    try:
        if len(loaded_res["baseline_cm"].classes) == 5:
            baseline_res = {
                "baseline_accuracy": get_else_nan(loaded_res["baseline_cm"], "ACC_Macro"),
                "baseline_f1_macro": get_else_nan(loaded_res["baseline_cm"], "F1_Macro"),
                "baseline_f1_micro": get_else_nan(loaded_res["baseline_cm"], "F1_Micro"),
                "baseline_mcc": get_else_nan(loaded_res["baseline_cm"], "Overall_MCC"),
                "train_size": loaded_res["train_size"],
                "representation": loaded_res["representation"],
                "target": loaded_res["target"],
            }
        else: 
            raise ValueError
    except Exception:
        baseline_res = {
                "baseline_accuracy": np.nan,
                "baseline_f1_macro": np.nan,
                "baseline_f1_micro": np.nan,
                "baseline_mcc": np.nan,
                "train_size": loaded_res["train_size"],
                "representation": loaded_res["representation"],
                "target": loaded_res["target"],
            }
    if 'skip_hyperopt' in filename:
        compiled_core_classification_data_skip_hyperopt.append(res)
        compiled_core_baseline_data_skip_hyperopt.append(baseline_res)
    else:
        compiled_core_classification_data.append(res)
        compiled_core_baseline_data.append(baseline_res)


In [407]:
compiled_core_classification_data = pd.DataFrame(compiled_core_classification_data)
compiled_core_baseline_data = pd.DataFrame(compiled_core_baseline_data)

compiled_core_classification_data_skip_hyperopt = pd.DataFrame(compiled_core_classification_data_skip_hyperopt)
compiled_core_baseline_data_skip_hyperopt = pd.DataFrame(compiled_core_baseline_data_skip_hyperopt)

In [408]:
compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_99655/2870106362.py:1: FutureWarning: ['mcc'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_core_classification_data_agg = compiled_core_classification_data.groupby(['target', 'representation', 'train_size']).agg(['mean', 'std'])


In [409]:
compiled_core_classification_data_agg

accuracy            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.717800  0.010701   
                                      50          0.727200  0.005581   
                                      100         0.721400  0.006000   
                                      200         0.806800  0.124025   
                                      500         0.736533  0.005478   
                                      1000        0.773067  0.008992   
                      clean_mofid     10          0.745229  0.043210   
                                      50          0.720600  0.008941   
                                      100         0.719733  0.002013   
                                      200         0.736400  0.014816   
                                      500         0.773462  0.059603   
                                      1000        0.756800  0.006882   
outputs.logKH_CO2_cat chemical_name_y 10          0.778433  0.006467   
                                      50          0.771600  0.006450   
                                      100         0.768800  0.012445   
                                      200         0.777000  0.007016   
                                      500         0.790175  0.020426   
                                      1000        0.837956  0.016460   
                      clean_mofid     10          0.764833  0.036884   
                                      50          0.772000  0.007811   
                                      100         0.768000  0.008641   
                                      200         0.782000  0.011323   
                                      500         0.790200  0.005362   
                                      1000        0.803400  0.010681   

                                                  f1_macro            \
                                                      mean       std   
target                representation  train_size                       
outputs.CH4DC_cat     chemical_name_y 10          0.149823  0.027937   
                                      50          0.130674  0.023026   
                                      100         0.153043  0.016027   
                                      200         0.132933  0.123551   
                                      500         0.249662  0.076365   
                                      1000        0.357930  0.034869   
                      clean_mofid     10          0.124279  0.026044   
                                      50          0.159902  0.047351   
                                      100         0.135585  0.020836   
                                      200         0.198593  0.056259   
                                      500         0.210406  0.108564   
                                      1000        0.310452  0.026149   
outputs.logKH_CO2_cat chemical_name_y 10          0.131826  0.009319   
                                      50          0.125988  0.010818   
                                      100         0.156510  0.024739   
                                      200         0.199429  0.050453   
                                      500         0.259241  0.153433   
                                      1000        0.327946  0.103090   
                      clean_mofid     10          0.163316  0.027357   
                                      50          0.168882  0.034863   
                                      100         0.162777  0.025103   
                                      200         0.208806  0.030507   
                                      500         0.224695  0.033614   
                                      1000        0.303062  0.038744   

                                                  f1_micro            
                                                      mean       std  
target            

In [410]:
compiled_core_baseline_data.groupby(['target',  "train_size"]).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_99655/2842005148.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_core_baseline_data.groupby(['target',  "train_size"]).agg(['mean', 'std'])


baseline_accuracy            \
                                              mean       std   
target                train_size                               
outputs.CH4DC_cat     10                       NaN       NaN   
                      50                  0.786994  0.025932   
                      100                 0.827163  0.010223   
                      200                 0.861877       NaN   
                      500                      NaN       NaN   
                      1000                     NaN       NaN   
outputs.logKH_CO2_cat 10                       NaN       NaN   
                      50                       NaN       NaN   
                      100                      NaN       NaN   
                      200                 0.803013       NaN   
                      500                 0.805891       NaN   
                      1000                     NaN       NaN   

                                 baseline_f1_macro            \
                                              mean       std   
target                train_size                               
outputs.CH4DC_cat     10                       NaN       NaN   
                      50                  0.411602  0.023767   
                      100                 0.536074  0.061020   
                      200                 0.647754       NaN   
                      500                      NaN       NaN   
                      1000                     NaN       NaN   
outputs.logKH_CO2_cat 10                       NaN       NaN   
                      50                       NaN       NaN   
                      100                      NaN       NaN   
                      200                 0.277860       NaN   
                      500                 0.365164       NaN   
                      1000                     NaN       NaN   

                                 baseline_f1_micro           baseline_mcc  \
                                              mean       std         mean   
target                train_size                                            
outputs.CH4DC_cat     10                       NaN       NaN          NaN   
                      50                  0.467484  0.064831     0.281864   
                      100                 0.567908  0.025558     0.419658   
                      200                 0.654693       NaN     0.540553   
                      500                      NaN       NaN          NaN   
                      1000                     NaN       NaN          NaN   
outputs.logKH_CO2_cat 10                       NaN       NaN          NaN   
                      50                       NaN       NaN          NaN   
                      100                      NaN       NaN          NaN   
                      200                 0.507532       NaN     0.214539   
                      500                 0.514727       NaN     0.236677   
                      1000                     NaN       NaN          NaN   

                                            
                                       std  
target                train_size            
outputs.CH4DC_cat     10               NaN  
                      50          0.084508  
                      100         0.036628  
                      200              NaN  
                      500              NaN  
                      1000             NaN  
outputs.logKH_CO2_cat 10               NaN  
                      50               NaN  
                      100              NaN  
                      200              NaN  
                      500              NaN  
                      1000             NaN

In [411]:
compiled_core_baseline_data_skip_hyperopt_agg =compiled_core_baseline_data_skip_hyperopt.groupby(['target',  "train_size"]).agg(['mean', 'std'])
compiled_core_baseline_data_skip_hyperopt_agg

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_99655/3125592674.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  compiled_core_baseline_data_skip_hyperopt_agg =compiled_core_baseline_data_skip_hyperopt.groupby(['target',  "train_size"]).agg(['mean', 'std'])


baseline_accuracy            \
                                              mean       std   
target                train_size                               
outputs.CH4DC_cat     10                  0.722651  0.010529   
                      50                  0.804115  0.010768   
                      100                 0.812859  0.009818   
                      200                 0.844959  0.004801   
                      500                 0.870937  0.003918   
                      1000                0.887257  0.004515   
outputs.logKH_CO2_cat 10                  0.737323  0.016218   
                      50                  0.769486  0.008209   
                      100                 0.784836  0.008639   
                      200                 0.791400  0.006498   
                      500                 0.808384  0.005787   
                      1000                0.819384  0.004695   

                                 baseline_f1_macro            \
                                              mean       std   
target                train_size                               
outputs.CH4DC_cat     10                  0.228574  0.021924   
                      50                  0.476998  0.025910   
                      100                 0.516830  0.027868   
                      200                 0.597759  0.015077   
                      500                 0.670434  0.010805   
                      1000                0.714128  0.014921   
outputs.logKH_CO2_cat 10                  0.196766  0.023445   
                      50                  0.239585  0.015373   
                      100                 0.252245  0.009660   
                      200                 0.301413  0.020863   
                      500                 0.327776  0.021308   
                      1000                0.413514  0.017588   

                                 baseline_f1_micro           baseline_mcc  \
                                              mean       std         mean   
target                train_size                                            
outputs.CH4DC_cat     10                  0.306628  0.026323     0.080307   
                      50                  0.510288  0.026919     0.346970   
                      100                 0.532147  0.024545     0.375949   
                      200                 0.612399  0.012004     0.483121   
                      500                 0.677341  0.009795     0.569829   
                      1000                0.718143  0.011288     0.625027   
outputs.logKH_CO2_cat 10                  0.343308  0.040546     0.018442   
                      50                  0.423715  0.020523     0.079805   
                      100                 0.462091  0.021597     0.142702   
                      200                 0.478499  0.016244     0.172343   
                      500                 0.520960  0.014467     0.240333   
                      1000                0.548461  0.011737     0.285674   

                                            
                                       std  
target                train_size            
outputs.CH4DC_cat     10          0.029159  
                      50          0.037424  
                      100         0.032717  
                      200         0.015312  
                      500         0.012482  
                      1000        0.014852  
outputs.logKH_CO2_cat 10          0.046429  
                      50          0.032316  
                      100         0.029400  
                      200         0.024734  
                      500         0.021738  
                      1000        0.017710

In [432]:
print(compile_table_baseline_row(compiled_core_baseline_data_skip_hyperopt_agg.loc['outputs.logKH_CO2_cat'].loc[1000:1000]))

\num{ 0.82 \pm 0.0 } &  \num{ 0.55 \pm 0.01  } & \num{ 0.41 \pm 0.02 }\\
